In [ ]:
############################################################################################
# IMPORTS
############################################################################################
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
#import tensorflow.keras as keras

from tensorflow.keras import layers
from tensorflow.keras import regularizers


from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Flatten

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical



from IPython import display
from PIL import Image


import pathlib
import shutil
import tempfile
import concurrent


import json
import glob





print(tf.__version__)
print(tf.config.list_physical_devices())
############################################################################################

In [ ]:
############################################################################################
# 1. CONSTANTS - PATHS
############################################################################################

DATA_FS251 = './data/iFood_2019'
CLASSES_FILE_NAME = '/formated_annot/classes_formated.csv'

TRAIN_INFO = '/annot/train_info.csv'
VAL_INFO = '/annot/val_info.csv'
TEST_INFO = '/annot/test_info.csv'

TRAIN_PICS_PATH = './data/iFood_2019/train_set/'
TEST_PICS_PATH = './data/iFood_2019/test_set/'
VAL_PICS_PATH = './data/iFood_2019/val_set/'

MODELS = './models/'

SEED = 111

#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [ ]:
############################################################################################
# IMPORTING DATA
############################################################################################
df_classes = pd.read_csv(DATA_FS251 + CLASSES_FILE_NAME)
df_train = pd.read_csv(DATA_FS251 + TRAIN_INFO, names=['file_name', 'class_num'])
df_validate = pd.read_csv(DATA_FS251 + VAL_INFO, names=['file_name', 'class_num'])
df_test = pd.read_csv(DATA_FS251 + TEST_INFO, names=['file_name'])

df_train.head(3)

In [ ]:
############################################################################################
# 2. CONSTANTS - MODEL
############################################################################################
training_history = dict()

N_TRAIN = len(df_train.iloc[:, 0])
EPOCHS = 10
BATCH_SIZE = 10              # 8 fit into GPU RAM, 64 fit into system RAM
RESIZE_TO = (256, 256)
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE

In [ ]:
############################################################################################
# import image files

train_pics = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PICS_PATH,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=1,
#    image_size=RESIZE_TO,
    shuffle=False,
    seed=SEED,
    validation_split=None,
    subset=None,
#    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    data_format=None,
    verbose=True
)
val_pics = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_PICS_PATH,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=1,
#    image_size=RESIZE_TO,
    shuffle=False,
    seed=SEED,
    validation_split=None,
    subset=None,
#    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    data_format=None,
    verbose=True
)
test_pics = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PICS_PATH,
    labels=None,
    label_mode=None,
    class_names=None,
    color_mode='rgb',
    batch_size=1,
#    image_size=RESIZE_TO,
    shuffle=False,
    seed=SEED,
    validation_split=None,
    subset=None,
#    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    data_format=None,
    verbose=True
)

In [ ]:
val_pics

In [ ]:

#tf.keras.preprocessing.image.load_img(train_pics[155])
#plt.imshow(train_pics)

In [ ]:
############################################################################################
# DEFINING THE MODEL
tf.keras.backend.clear_session()
model_supclass = tf.keras.models.Sequential([
# Zero's layer - resizing:
        tf.keras.layers.Resizing(
        height = 256,
        width = 256,
        interpolation='bilinear',
        crop_to_aspect_ratio=False,
        pad_to_aspect_ratio=False,
        fill_mode='constant',
        fill_value=0.0,
        data_format=None),
# First conv. layers set 256p:
        tf.keras.layers.Conv2D(
                filters=32,
                kernel_size = (3,3),
                strides=(1, 1),
                padding='valid',
                data_format=None,
                dilation_rate=(1, 1),
                groups=1,
                activation='relu',
                use_bias=True,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None,
                kernel_constraint=None,
                bias_constraint=None
        ),
        tf.keras.layers.Conv2D(
                filters=32,
                kernel_size = (3,3),
                strides=(1, 1),
                padding='valid',
                data_format=None,
                dilation_rate=(1, 1),
                groups=1,
                activation='relu',
                use_bias=True,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None,
                kernel_constraint=None,
                bias_constraint=None
       ),
# Second conv. layers set 256->128:
        tf.keras.layers.MaxPooling2D(
                pool_size=(2, 2),
                strides=2,
                padding='valid',
                data_format=None,
                name='256_to_128_MaxPool'
        ),
        tf.keras.layers.Conv2D(
                filters=64,
                kernel_size = (3,3),
                strides=(1, 1),
                padding='valid',
                data_format=None,
                dilation_rate=(1, 1),
                groups=1,
                activation='relu',
                use_bias=True,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None,
                kernel_constraint=None,
                bias_constraint=None
        ),
#        tf.keras.layers.Conv2D(
#                filters=64,
#                kernel_size = (3,3),
#                strides=(1, 1),
#                padding='valid',
#                data_format=None,
#                dilation_rate=(1, 1),
#                groups=1,
#                activation='relu',
#                use_bias=True,
#                kernel_initializer='glorot_uniform',
#                bias_initializer='zeros',
#                kernel_regularizer=None,
#                bias_regularizer=None,
#                activity_regularizer=None,
#                kernel_constraint=None,
#                bias_constraint=None
#        ),
# Third conv. layers set 128->64:
        tf.keras.layers.MaxPooling2D(
                pool_size=(2, 2),
                strides=2,
                padding='valid',
                data_format=None,
                name='128_to_64_MaxPool'
        ),
        tf.keras.layers.Conv2D(
                filters=128,
                kernel_size = (3,3),
                strides=(1, 1),
                padding='valid',
                data_format=None,
                dilation_rate=(1, 1),
                groups=1,
                activation='relu',
                use_bias=True,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None,
                kernel_constraint=None,
                bias_constraint=None
        ),
#        tf.keras.layers.Conv2D(
#                filters=128,
#                kernel_size = (3,3),
#                strides=(1, 1),
#                padding='valid',
#                data_format=None,
#                dilation_rate=(1, 1),
#                groups=1,
#                activation='relu',
#                use_bias=True,
#                kernel_initializer='glorot_uniform',
#                bias_initializer='zeros',
#                kernel_regularizer=None,
#                bias_regularizer=None,
#                activity_regularizer=None,
#                kernel_constraint=None,
#                bias_constraint=None
#        ),
# Fourth conv. layers set 64->32:
        tf.keras.layers.MaxPooling2D(
                pool_size=(2, 2),
                strides=2,
                padding='valid',
                data_format=None,
                name='64_to_32_MaxPool'
        ),
        tf.keras.layers.Conv2D(
                filters=256,
                kernel_size = (3,3),
                strides=(1, 1),
                padding='valid',
                data_format=None,
                dilation_rate=(1, 1),
                groups=1,
                activation='relu',
                use_bias=True,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None,
                kernel_constraint=None,
                bias_constraint=None
        ),
#        tf.keras.layers.Conv2D(
#                filters=256,
#                kernel_size = (3,3),
#                strides=(1, 1),
#                padding='valid',
#                data_format=None,
#                dilation_rate=(1, 1),
#                groups=1,
#                activation='relu',
#                use_bias=True,
#                kernel_initializer='glorot_uniform',
#                bias_initializer='zeros',
#                kernel_regularizer=None,
#                activity_regularizer=None,
#                kernel_constraint=None,
#                bias_constraint=None
#        ),
# Third conv. layers set 32->16:
        tf.keras.layers.MaxPooling2D(
                pool_size=(2, 2),
                strides=2,
                padding='valid',
                data_format=None,
                name='32_to_16_MaxPool'
        ),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(2048,kernel_initializer = 'uniform', activation='relu'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(1024,kernel_initializer = 'uniform', activation='relu'),
#        tf.keras.layers.Dense(units=251,activation='softmax')
        tf.keras.layers.Dense(units=251,activation=tf.keras.activations.sigmoid)
])



model_supclass.summary()


In [ ]:
############################################################################################
# COMPILING
############################################################################################
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.00001,
    decay_steps=STEPS_PER_EPOCH*100,
    decay_rate=1,
    staircase=False)

model_supclass.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule),
        loss = tf.keras.losses.categorical_crossentropy,
        metrics = [tf.keras.metrics.CategoricalAccuracy()] 
        )

#model_supclass.summary()

In [ ]:
############################################################################################
# FITTING
############################################################################################
with tf.device('/gpu:0'):
    model_history = model_supclass.fit(
        train_pics,
        epochs = EPOCHS, 
        batch_size = BATCH_SIZE,
        validation_data = val_pics
    )

model_supclass.summary()



In [ ]:
############################################################################################
# SAVING MODEL
############################################################################################

os.makedirs( os.path.dirname(MODELS), exist_ok=True)
model_supclass.save(MODELS + "custom_251_v0.1.h5")
model_supclass = load_model(MODELS + "custom_251_v0.1.h5")

STOP_HERE

In [ ]:
############################################################################################
# EVALUATION
############################################################################################

model_history.history.keys()
pd.DataFrame(model_history.history).plot()
plt.legend(bbox_to_anchor = [1, 1.02])
plt.plot(model_history.history['val_loss'])
plt.title('Validation Loss')
plt.xlabel('epochs')

# Print out the score
score = model_supclass.evaluate(val_pics , batch_size=10, verbose=1)
print(score, model_supclass.metrics_names)

############################################################################################
# PREDICTION
############################################################################################







y_hat=model.predict(new_X)




